In [3]:
import pandas as pd
import os
from statsmodels.stats.multitest import multipletests


In [4]:
root_dir = "gtex_fdr_results"
tissues = [
    "Adipose_Tissue", "Adrenal_Gland", "Bladder", "Blood", "Blood_Vessel",
    "Brain", "Breast", "Cervix_Uteri", "Colon", "Esophagus",
    "Fallopian_Tube", "Heart", "Kidney", "Liver", "Lung",
    "Muscle", "Nerve", "Ovary", "Pancreas", "Pituitary",
    "Prostate", "Salivary_Gland", "Skin", "Small_Intestine", "Spleen",
    "Stomach", "Testis", "Thyroid", "Uterus", "Vagina"
    ]
groundtruth_dir = "batch_wise_fdr_grns"
approx_dir = "random_targets_wasserstein_up_to_100"
approx_file = "fdr_grn_nontf_100_numtf_-1.csv"
groundtruth_file = "groundtruth_grn.csv"

In [5]:
all_distance_df = pd.DataFrame(index=tissues, columns=tissues)
gt_distance_df = pd.DataFrame(index=tissues, columns=tissues)
approx_distance_df = pd.DataFrame(index=tissues, columns=tissues)
highest_expressed = True

for first in range(len(tissues)):
    print("Setting tissue 1 = ", tissues[first])
    # Read first tissue.
    gt_path = os.path.join(root_dir, tissues[first], groundtruth_dir, groundtruth_file)
    approx_path = os.path.join(root_dir, tissues[first], approx_dir, approx_file)
    first_gt = pd.read_csv(gt_path)
    first_approx = pd.read_csv(approx_path)
    
    if highest_expressed == True:
        # Subset to highest expressed genes.
        high_exp_file = os.path.join("gtex", tissues[first], f"{tissues[first]}_highly_expressed_targets.csv")
        high_df = pd.read_csv(high_exp_file)
        top_targets = list(high_df['target'])[:1000]
        first_gt = first_gt[first_gt['target'].isin(top_targets)]
        first_approx = first_approx[first_approx['target'].isin(top_targets)]    
    
    # Compute BH correction on approx. and groundtruths.
    _, gt_pvals_bh, _, _ = multipletests(first_gt['pvalue'], method='fdr_bh')
    first_gt['pvalue_bh'] = gt_pvals_bh
    # Compute BH correction on approx. and groundtruths.
    _, approx_pvals_bh, _, _ = multipletests(first_approx['pvalue'], method='fdr_bh')
    first_approx['pvalue_bh'] = approx_pvals_bh
    
    for second in range(first+1, len(tissues)):
        # Read second tissue.
        gt_path = os.path.join(root_dir, tissues[second], groundtruth_dir, groundtruth_file)
        approx_path = os.path.join(root_dir, tissues[second], approx_dir, approx_file)
        second_gt = pd.read_csv(gt_path)
        second_approx = pd.read_csv(approx_path)
        
        if highest_expressed == True:
            # Subset to highest expressed genes.
            high_exp_file = os.path.join("gtex", tissues[second], f"{tissues[second]}_highly_expressed_targets.csv")
            high_df = pd.read_csv(high_exp_file)
            top_targets = list(high_df['target'])[:1000]
            second_gt = second_gt[second_gt['target'].isin(top_targets)]
            second_approx = second_approx[second_approx['target'].isin(top_targets)]    
        
        _, gt_pvals_bh, _, _ = multipletests(second_gt['pvalue'], method='fdr_bh')
        second_gt['pvalue_bh'] = gt_pvals_bh
        # Compute BH correction on approx. and groundtruths.
        _, approx_pvals_bh, _, _ = multipletests(second_approx['pvalue'], method='fdr_bh')
        second_approx['pvalue_bh'] = approx_pvals_bh
        
        cols = ['TF', 'target']

        ### Compute jaccard distance on all edges (full GRN).
        # Convert to sets of tuples (TF, target)
        edges_all1 = set(first_approx[cols].itertuples(index=False, name=None))
        edges_all2 = set(second_approx[cols].itertuples(index=False, name=None))

        # Compute intersection and union
        all_intersection = len(edges_all1 & edges_all2)
        all_union = len(edges_all1 | edges_all2)

        # Compute Jaccard similarity and distance
        sim_all = all_intersection / all_union if all_union != 0 else 0
        dist_all = 1 - sim_all
        
        ### Compute jaccard distance on only significant groundtruth edges.
        signif_gt1 = first_gt[first_gt["pvalue_bh"]<=0.05]
        signif_gt2 = second_gt[second_gt["pvalue_bh"]<=0.05]
        
        edges_gt1 = set(signif_gt1[cols].itertuples(index=False, name=None))
        edges_gt2 = set(signif_gt2[cols].itertuples(index=False, name=None))

        # Compute intersection and union
        gt_intersection = len(edges_gt1 & edges_gt2)
        gt_union = len(edges_gt1 | edges_gt2)

        # Compute Jaccard similarity and distance
        sim_gt = gt_intersection / gt_union if gt_union != 0 else 0
        dist_gt = 1 - sim_gt
        
        ### Compute jaccard distance only on signif. approx. edges.
        signif_approx1 = first_approx[first_approx["pvalue_bh"]<=0.05]
        signif_approx2 = second_approx[second_approx["pvalue_bh"]<=0.05]
        
        edges_approx1 = set(signif_approx1[cols].itertuples(index=False, name=None))
        edges_approx2 = set(signif_approx2[cols].itertuples(index=False, name=None))

        # Compute intersection and union
        approx_intersection = len(edges_approx1 & edges_approx2)
        approx_union = len(edges_approx1 | edges_approx2)

        # Compute Jaccard similarity and distance
        sim_approx = approx_intersection / approx_union if approx_union != 0 else 0
        dist_approx = 1 - sim_approx
        
        # Save distances in results dataframe.
        all_distance_df.loc[tissues[first], tissues[second]] = dist_all
        all_distance_df.loc[tissues[second], tissues[first]] = dist_all
        
        gt_distance_df.loc[tissues[first], tissues[second]] = dist_gt
        gt_distance_df.loc[tissues[second], tissues[first]] = dist_gt
        
        approx_distance_df.loc[tissues[first], tissues[second]] = dist_approx
        approx_distance_df.loc[tissues[second], tissues[first]] = dist_approx
    

Setting tissue 1 =  Adipose_Tissue
Setting tissue 1 =  Adrenal_Gland
Setting tissue 1 =  Bladder
Setting tissue 1 =  Blood
Setting tissue 1 =  Blood_Vessel
Setting tissue 1 =  Brain
Setting tissue 1 =  Breast
Setting tissue 1 =  Cervix_Uteri
Setting tissue 1 =  Colon
Setting tissue 1 =  Esophagus
Setting tissue 1 =  Fallopian_Tube
Setting tissue 1 =  Heart
Setting tissue 1 =  Kidney
Setting tissue 1 =  Liver
Setting tissue 1 =  Lung
Setting tissue 1 =  Muscle
Setting tissue 1 =  Nerve
Setting tissue 1 =  Ovary
Setting tissue 1 =  Pancreas
Setting tissue 1 =  Pituitary
Setting tissue 1 =  Prostate
Setting tissue 1 =  Salivary_Gland
Setting tissue 1 =  Skin
Setting tissue 1 =  Small_Intestine
Setting tissue 1 =  Spleen
Setting tissue 1 =  Stomach
Setting tissue 1 =  Testis
Setting tissue 1 =  Thyroid
Setting tissue 1 =  Uterus
Setting tissue 1 =  Vagina


In [6]:
print(all_distance_df.shape)
print(gt_distance_df.shape)
print(approx_distance_df.shape)
all_distance_df.to_csv("tissue_distances_all_edges_top1000.csv", index=True)
gt_distance_df.to_csv("tissue_distances_significant_groundtruth_top1000.csv", index=True)
approx_distance_df.to_csv("tissue_distances_significant_approx_top1000.csv", index=True)

(30, 30)
(30, 30)
(30, 30)
